<a href="https://colab.research.google.com/github/OMatheusWander/Cienciadedadosparaeconomistas/blob/main/STATA_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Class utilizing COVID-19 data to do calculus with  Mincer earnings function
#/*
#forvalues i = 5/11 {
#cls
#clear all
# import delimited C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_`i'.csv, clear
# keep if uf == 42
# keep if v1022 == 1
# keep if v1023 == 1
# save C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_`i'.dta, replace
}

cls
clear all
use C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_5.dta
forvalues i = 6/11 {
 append using C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_`i'.dta
 save C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\data_COVID.dta, replace
}
*/

cls
clear all
*import delimited "https://petterini.ufsc.br/files/2023/09/data_covid_sc.txt", ///
*       delimiter(comma) varnames(1) encoding(utf8)
*save "C:\Users\Usuario\Desktop\data_covid_sc.dta", replace
use "C:\Users\CPF\Downloads\data_covid_sc.dta"

egen id = group(v1008 estrato upa)

rename v1013 mes
drop if a001a != 1
*keep if a001a == 1
sort id mes
order id mes
bysort id: gen times1 = _n
bysort id: gen times2 = _N
order id mes times1 times2
tab times2
drop if times2 == 1
rename a002 idade
gen homem = cond(a003 == 1, 1, 0)
gen branco = cond(a004 == 1, 1, 0)
rename a005 edu
tab edu, m
rename c008 horas
rename c01012 w
order id mes idade homem branco edu horas w
label variable id "id"
label variable mes "mes"
label variable idade "age"
label variable homem "=1 if man"
label variable branco "=1 if white"
label variable edu "categorical see dic"
label variable horas "week work"
label variable w "month wage"
drop times1-a007a
g w2 = w/((30/7)*horas)
gen lnw2 = ln(w2)
sort id mes
tsset id mes
tsfill, full

sort id mes
bysort id: gen ok = cond(_n == 1 & idade != . & homem != . ///
                               & branco != . & edu != .,1,0)

bysort id: replace ok = ok[_n-1] if _n > 1
drop if ok == 0
drop ok

bysort id: egen idade2 = max(idade)
drop if idade2 == .
bysort id: gen ok = cond(idade == . | idade == idade2 | idade == idade2-1 ,1,0)
bysort id: egen ok2 = mean(ok)
drop if ok2 != 1
drop ok ok2 idade2

bysort id: replace idade = idade[_n-1] if idade == .

bysort id: replace homem = homem[_n-1] if homem == .
bysort id: egen homem2 = mean(homem)
drop if homem != homem2
drop homem2

bysort id: replace branco = branco[_n-1] if branco == .
bysort id: egen branco2 = mean(branco)
drop if branco != branco2
drop branco2

bysort id: egen edu2 = max(edu)
drop if edu2 == .
bysort id: gen ok = cond(edu == . | edu == edu2 | edu == edu2-1 ,1,0)
bysort id: egen ok2 = mean(ok)
drop if ok2 != 1
drop ok ok2 edu2

*tw (scatter lnw edu) (lfit lnw edu)
*tw (scatter lnw idade) (qfit lnw idade)

sort id mes

tsset id mes
tsfill, full

gen idade2 = idade^2

reg lnw2 idade idade2 i.edu homem branco

*reg lnw2 idade idade2 i.edu i.id

tab edu, generate(dedu)

sort id mes

foreach var of varlist lnw2 idade idade2 dedu2-dedu8 {
bysort id: egen m`var' = mean(`var') if lnw2 != .
replace m`var' = `var'-m`var'
}

xtset id mes
xtreg lnw2 idade idade2 i.edu, fe
xtreg lnw2 idade idade2 dedu2-dedu8, fe

reg mlnw2 midade midade2 mdedu2 mdedu3 mdedu4 mdedu5 mdedu6 mdedu7 mdedu8

g emp = cond(lnw2 != .,1,0)

gen xbprobit = .
local t = 5
while `t' <= 11 {
 probit emp idade idade2 homem branco if mes == `t'
 predict xb, xb
 replace xbprobit = normalden(xb)/normal(xb) if mes == `t'
 drop xb
local t = `t' + 1
}

reg mlnw2 midade midade2 mdedu2 mdedu3 mdedu4 mdedu5 mdedu6 mdedu7 mdedu8
estimates store reg1
reg mlnw2 midade midade2 mdedu2 mdedu3 mdedu4 mdedu5 mdedu6 mdedu7 mdedu8 xbprobit
estimates store reg2

esttab reg1 reg2, se(3) b(3) star(* 0.1 ** 0.05 *** 0.01)




In [ ]:
 histogram w, percent fcolor(blue)
(bin=45, start=5, width=2599.8889)

. histogram w if w <20000, , percent fcolor(blue)
invalid 'percent'
r(198);

. histogram w if w <20000, percent fcolor(blue)
(bin=45, start=5, width=435.44444)

. kdensity w

. tw (kdensity w)(hist w)

. tw (kdensity w if w <20000 ) (hist w if w<200000)

. tw (kdensity w if w <20000 & homem ==1 ) (hist w if w<200000 & homem ==1)

. scatter lnw2 edu
 aaplot lnw2 edu

# graphics commands